In [14]:
import gzip
from collections import defaultdict

def readjson(f):
    
  for l in open(f):
    yield eval(l)

In [15]:
categories_fulldata=[]
headlines_fulldata=[]
sdescriptions_fulldata=[]
for l in readjson("News_Category_Dataset.json"):
    category,headline,sdescription=l['category'],l['headline'],l['short_description']
    categories_fulldata.append(category)
    headlines_fulldata.append(headline)
    sdescriptions_fulldata.append(sdescription)
    
    
    

In [16]:
categories_train=categories_fulldata[:50000]
headlines_train=headlines_fulldata[:50000]
sdescriptions_train=sdescriptions_fulldata[:50000]

In [18]:
categories_validation=categories_fulldata[50000:75000]
headlines_validation=headlines_fulldata[50000:75000]
sdescriptions_validation=sdescriptions_fulldata[50000:75000]

In [20]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chapsujith/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


In [26]:
sample = sdescriptions_train[0]

In [27]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [28]:
print('original document: ')
words = []
for word in sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(sample))

original document: 
['She', 'left', 'her', 'husband.', 'He', 'killed', 'their', 'children.', 'Just', 'another', 'day', 'in', 'America.']


 tokenized and lemmatized document: 
[u'leav', u'husband', u'kill', u'children', u'america']


In [30]:
processed_headlines=map(preprocess,headlines_train)

In [31]:
processed_headlines

[[u'mass', u'shoot', u'texa', u'week'],
 [u'smith', u'join', u'diplo', u'nicki', u'world', u'offici', u'song'],
 [u'hugh', u'grant', u'marri', u'time'],
 [u'carrey',
  u'blast',
  u'castrato',
  u'adam',
  u'schiff',
  u'democrat',
  u'artwork'],
 [u'julianna',
  u'marguli',
  u'use',
  u'donald',
  u'trump',
  u'poop',
  u'bag',
  u'pick'],
 [u'morgan',
  u'freeman',
  u'devast',
  u'sexual',
  u'harass',
  u'claim',
  u'undermin',
  u'legaci'],
 [u'donald', u'trump', u'lovin', u'mcdonald', u'jingl', u'tonight'],
 [u'watch', u'amazon', u'prime', u'week'],
 [u'mike',
  u'myer',
  u'reveal',
  u'like',
  u'fourth',
  u'austin',
  u'power',
  u'film'],
 [u'watch', u'hulu', u'week'],
 [u'justin', u'timberlak', u'visit', u'texa', u'school', u'shoot', u'victim'],
 [u'south',
  u'korean',
  u'presid',
  u'meet',
  u'north',
  u'korea',
  u'jong',
  u'talk',
  u'trump',
  u'summit'],
 [u'life',
  u'risk',
  u'remot',
  u'oyster',
  u'grow',
  u'region',
  u'call',
  u'robot'],
 [u'trump', u'c

In [33]:
dictionary = gensim.corpora.Dictionary(processed_headlines)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

(2160, u'fawn')
(14145, u'headband')
(14381, u'mustachio')
(549, u'woodi')
(14884, u'galactica')
(14902, u'mccourti')
(4199, u'foul')
(6319, u'elvi')
(15779, u'askew')
(15972, u'prefix')
(13095, u'jihad')


In [34]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [36]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_headlines]

In [38]:
bow_corpus[50]

[(13, 1), (200, 1), (242, 1), (243, 1), (244, 1), (245, 1)]

In [40]:
dictionary[0]

u'mass'

In [41]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5794203897693117),
 (1, 0.4340631596360102),
 (2, 0.5044666304490711),
 (3, 0.47050462710768615)]


In [42]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [43]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.020*"attack" + 0.018*"kill" + 0.016*"polic" + 0.012*"north" + 0.012*"trump" + 0.011*"say" + 0.009*"korea" + 0.008*"syria" + 0.007*"home" + 0.007*"american"
Topic: 1 
Words: 0.015*"year" + 0.011*"video" + 0.010*"peopl" + 0.009*"news" + 0.009*"trump" + 0.007*"thing" + 0.007*"need" + 0.007*"tran" + 0.007*"love" + 0.006*"fake"
Topic: 2 
Words: 0.016*"fight" + 0.012*"trump" + 0.011*"dead" + 0.011*"travel" + 0.009*"black" + 0.009*"lose" + 0.009*"nation" + 0.008*"peopl" + 0.008*"world" + 0.008*"star"
Topic: 3 
Words: 0.107*"trump" + 0.056*"donald" + 0.017*"court" + 0.013*"voter" + 0.009*"say" + 0.009*"suprem" + 0.009*"stephen" + 0.008*"right" + 0.008*"colbert" + 0.008*"presid"
Topic: 4 
Words: 0.063*"trump" + 0.028*"white" + 0.024*"donald" + 0.021*"hous" + 0.016*"tweet" + 0.012*"jam" + 0.011*"week" + 0.009*"say" + 0.009*"death" + 0.008*"interview"
Topic: 5 
Words: 0.038*"trump" + 0.017*"donald" + 0.016*"obama" + 0.013*"say" + 0.010*"mike" + 0.010*"watch" + 0.010*"penc" + 0.

In [46]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=30, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.014*"cancer" + 0.011*"attorney" + 0.010*"gener" + 0.010*"itali" + 0.009*"georg" + 0.009*"pay" + 0.008*"chrissi" + 0.008*"teigen" + 0.008*"tribut" + 0.008*"babi"
Topic: 1 Word: 0.018*"aleppo" + 0.011*"trump" + 0.010*"roger" + 0.010*"hair" + 0.009*"medic" + 0.009*"thank" + 0.008*"tale" + 0.008*"fashion" + 0.008*"deplor" + 0.008*"ail"
Topic: 2 Word: 0.015*"obama" + 0.014*"today" + 0.013*"trump" + 0.011*"donald" + 0.009*"human" + 0.009*"paul" + 0.009*"barack" + 0.008*"michel" + 0.007*"flynn" + 0.007*"design"
Topic: 3 Word: 0.022*"korea" + 0.022*"north" + 0.014*"roundup" + 0.012*"news" + 0.011*"probabl" + 0.010*"rememb" + 0.009*"conflict" + 0.009*"nuclear" + 0.008*"south" + 0.008*"east"
Topic: 4 Word: 0.013*"parent" + 0.010*"birthday" + 0.010*"teach" + 0.009*"doubl" + 0.009*"rock" + 0.009*"queen" + 0.008*"sick" + 0.008*"millenni" + 0.008*"kid" + 0.008*"land"
Topic: 5 Word: 0.021*"penc" + 0.020*"mike" + 0.014*"thing" + 0.011*"matter" + 0.011*"trump" + 0.011*"mom" + 0.010*"sa

In [48]:
count=0
for i in categories_train:
    if(i=='POLITICS'):
        
        count+=1
print count

17519
